<a href="https://colab.research.google.com/github/VishalPrem1994/AiGenPlayground/blob/main/Udemy_Course_1_Running_On_Trained_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown Download Important Stuff
%pip install -q -U --pre triton
%pip install -q accelerate transformers ftfy bitsandbytes==0.35.0 gradio natsort safetensors xformers
%pip install -qq git+https://github.com/VishalPrem1994/diffusers


In [ ]:
#@markdown Specify the weights directory to use 
WEIGHTS_DIR = "" #@param {type:"string"}
WEIGHTS_DIR = "/content/drive/MyDrive/" + WEIGHTS_DIR
print(f"[*] WEIGHTS_DIR={WEIGHTS_DIR}")
from google.colab import drive
drive.mount('/content/drive')

import torch
from torch import autocast
from diffusers import StableDiffusionPipeline, DDIMScheduler
from IPython.display import display

model_path = WEIGHTS_DIR             # If you want to use previously trained model saved in gdrive, replace this with the full path of model in gdrive
torch.cuda.empty_cache()
pipe = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=torch.float16).to("cuda")
pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
pipe.enable_xformers_memory_efficient_attention()
g_cuda = None




In [ ]:
g_cuda = torch.Generator(device='cuda')
seed = 52362234 #@param {type:"number"}
g_cuda.manual_seed(seed)
#@title Run for generating images.

prompt = "" #@param {type:"string"}
negative_prompt = "" #@param {type:"string"}
num_samples = 5 #@param {type:"number"}
guidance_scale = 8 #@param {type:"number"}
num_inference_steps = 50 #@param {type:"number"}
height = 512 #@param {type:"number"}
width = 512 #@param {type:"number"}

with autocast("cuda"), torch.inference_mode():
    images = pipe(
        prompt,
        height=height,
        width=width,
        negative_prompt=negative_prompt,
        num_images_per_prompt=num_samples,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        generator=g_cuda
    ).images

for img in images:
    display(img)